In [1]:
%reset -f

In [2]:
import os
import time
import scipy
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.image import ImageDataGenerator
tfk = tf.keras
tfkl = tf.keras.layers

In [3]:
onColab = False
if onColab:
  from google.colab import drive
  drive.mount('/gdrive')
  %cd /gdrive/MyDrive/University/ANN/CHALLENGE1/
  !ls

In [4]:
doUnzipData = False
batch_size = 64
validation_split = 0.3

if doUnzipData:
  !unzip dataset.zip

In [5]:
seed=20

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [6]:
dataset_path = './divided2'
img_h, img_w = (256, 256)
test_path = './test'


def load_data(dontUseFun=True, fun=None, isTest = False):
  """
  load data (train, val, test)
  dontUseFun: if it's true then the data is not preprocessed. If it's false, the data is preprocessed with fun (which must not be none)
  """
  image_generator = ImageDataGenerator(preprocessing_function = fun, validation_split=validation_split, 
                                          rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True, 
                                          fill_mode='reflect')  
  if dontUseFun:
    image_generator = ImageDataGenerator(validation_split=validation_split, 
                                          rotation_range=30,
                                          height_shift_range=50,
                                          width_shift_range=50,
                                          zoom_range=0.3,
                                          horizontal_flip=True,
                                          vertical_flip=True, 
                                          fill_mode='reflect')  


  # Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
  train_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                subset='training',
                                                batch_size=batch_size,
                                                shuffle=True,
                                                seed=seed)

  valid_gen = image_generator.flow_from_directory(directory=dataset_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                subset='validation',
                                                batch_size=batch_size,
                                                shuffle=False,
                                                seed=seed)
  test_gen = None
  if isTest:
    test_image_gen = ImageDataGenerator(preprocessing_function = fun)
    if dontUseFun:
      test_image_gen = ImageDataGenerator(rotation_range=30)
    
    test_gen = test_image_gen.flow_from_directory(directory=test_path,
                                                target_size=(img_h, img_w),
                                                color_mode='rgb',
                                                classes=None, # can be set to None
                                                class_mode='categorical',
                                                batch_size=batch_size,
                                                shuffle=True, # sometimes, instead of train, we use the test for training
                                                seed=seed)
  return train_gen, valid_gen, test_gen


In [7]:
# we need this ONLY for the labels
train_gen = load_data(True, None, False)[0]
labels = list(train_gen.class_indices.keys())


Found 12367 images belonging to 27 classes.
Found 5282 images belonging to 27 classes.


In [8]:
# labels = ['Apple', 'Apple_ill', 'Blueberry', 'Cherry', 'Cherry_ill', 'Corn_in', 'Corn_out', 'Grape', 'Grape_ill', 'Orange', 'Peach', 'Peach_ill', 'Pepper', 'Pepper_ill', 'Potato', 'Potato_ill', 'Raspberry', 'Soybean', 'Squash', 'Strawberry', 'Strawberry_ill', 'Tomato_green', 'Tomato_others']

In [9]:
create_test_set = False

# IF YOU CHANGE NET GO CHECK THE FINE TUNING CODE IF IT IS ALRIGHT!

model_dir = 'incepti_resnet_v2_divided2_ds'

# Highly dependent on the network chosen:
n_not_trainable_layers = 696

net = tfk.applications.InceptionResNetV2


def tl_preprocess(x):
  return tf.keras.applications.inception_resnet_v2.preprocess_input(x)

def compute_weight(val_split):
  w = []
  for i in range(len(labels)):
    class_imgs = next(os.walk('{}/{}/'.format('./divided2', labels[i])))[2]
    w.append(len(class_imgs) * (1 - val_split))

  tot = sum(w)
  for i in range(len(w)):
    w[i] = tot / w[i]
  m = max(w)
  w = np.array(w) / m
  w[np.argmax(w)] = 0.99999999999999
  d = {}
  for i in range(len(w)):
    d[i] = w[i]
  return d

# weights computed for validation split = 0.3
class_weight = compute_weight(validation_split)

# mappings = {
#     0: 0, 1: 0,
#     2: 1,
#     3: 2, 4: 2,
#     5: 3, 6: 3,
#     7: 4, 8: 4,
#     9: 5,
#     10: 6, 11: 6,
#     12: 7, 13: 7,
#     14: 8, 15: 8,
#     16: 9, 
#     17: 10,
#     18: 11,
#     19: 12, 20: 12,
#     21: 13, 22: 13
# }

drop_out_last_dense = 0
fc_dense_n_neurons = 0

comment = 'EfficientNetB1, transfer learning, no fine tuning. GAP used. no hidden dense layer with drop out'
comment += '\nnot trainable layers: ' + str(n_not_trainable_layers)
comment += ' drop_out_last_dense ' + str(drop_out_last_dense)
comment += ' fc_dense_n_neurons ' + str(fc_dense_n_neurons)
comment += '\nvalidation_split = ' + str(validation_split) 
comment += '. batch_size = ' + str(batch_size) 
comment += '. seed = ' + str(seed)
comment += '\nweigths: '
comment += str(class_weight)
print(comment)

EfficientNetB1, transfer learning, no fine tuning. GAP used. no hidden dense layer with drop out
not trainable layers: 696 drop_out_last_dense 0 fc_dense_n_neurons 0
validation_split = 0.3. batch_size = 64. seed = 20
weigths: {0: 0.15611814345991562, 1: 0.15744680851063828, 2: 0.16554809843400448, 3: 0.2846153846153846, 4: 0.23197492163009406, 5: 0.2072829131652661, 6: 0.08820023837902265, 7: 0.24183006535947713, 8: 0.0647419072615923, 9: 0.05124653739612189, 10: 0.28030303030303033, 11: 0.2901960784313725, 12: 0.10292072322670377, 13: 0.164079822616408, 14: 0.2442244224422442, 15: 0.4157303370786517, 16: 0.1182108626198083, 17: 0.28136882129277563, 18: 0.056661562021439515, 19: 0.25517241379310346, 20: 0.13909774436090225, 21: 0.99999999999999, 22: 0.22289156626506024, 23: 0.26618705035971224, 24: 0.7708333333333335, 25: 0.1488933601609658, 26: 0.01443620756925478}


In [10]:
if create_test_set:
  import shutil
  os.mkdir(test_path)
  for i in range(len(labels)):
      class_imgs = next(os.walk('{}/{}/'.format(dataset_dir, labels[i])))[2]
      # In this way we always get the same data. 
      # TODO: randomize using seed the data we take
      class_imgs.sort()
      for j in range(5):
        class_img = class_imgs[j]
        dest_dir = './test/' + labels[i] + '/'
        if not os.path.exists(dest_dir):
          os.mkdir(dest_dir)
        shutil.move('{}/{}/'.format(dataset_dir, labels[i]) + class_img, dest_dir)


In [11]:
def preprocess(x):
  # x = clean_image(x)
  if tl_preprocess:
    return tl_preprocess(np.uint8(x))
  return x

In [12]:
train_gen, valid_gen, test_gen = load_data(False, preprocess)

Found 12367 images belonging to 27 classes.
Found 5282 images belonging to 27 classes.


In [13]:
def get_next_batch(generator):
  batch = next(generator)

  image = batch[0]
  target = batch[1]

  print("(Input) image shape:", image.shape)
  print("Target shape:",target.shape)

  # Visualize only the first sample
  image = image[0]
  target = target[0]
  target_idx = np.argmax(target)
  print()
  print("Categorical label:", target)
  print("Label:", target_idx)
  print("Class name:", labels[target_idx])
  fig = plt.figure(figsize=(6, 4))
  plt.imshow(image)
  plt.title("image which will be fed to the cnn")
  return batch

# Get a sample from dataset and show info
# _ = get_next_batch(train_gen)

In [14]:
supernet =  net(
    weights="imagenet",
    input_shape=(img_h, img_w, 3),
    include_top=False
    )

In [15]:
print(len(supernet.layers))

780


In [16]:
# let last block trainable
for i, layer in enumerate(supernet.layers[:n_not_trainable_layers]):
  layer.trainable=False
for i, layer in enumerate(supernet.layers):
  print(i, layer.name, layer.trainable)
# supernet.summary()

# Now we use the supernet as layer of our network
inputs = tfk.Input(shape=(img_h, img_w, 3))
x = supernet(inputs)

# Now we build our classifier (which will be actually trained)
x = tfkl.GlobalAveragePooling2D(name="avg_pool")(x)
x = tfkl.BatchNormalization()(x)
outputs = tfkl.Dense(
    len(labels), 
    activation='softmax',
    kernel_initializer = tfk.initializers.GlorotUniform(seed))(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')

model.summary()

0 input_1 False
1 conv2d False
2 batch_normalization False
3 activation False
4 conv2d_1 False
5 batch_normalization_1 False
6 activation_1 False
7 conv2d_2 False
8 batch_normalization_2 False
9 activation_2 False
10 max_pooling2d False
11 conv2d_3 False
12 batch_normalization_3 False
13 activation_3 False
14 conv2d_4 False
15 batch_normalization_4 False
16 activation_4 False
17 max_pooling2d_1 False
18 conv2d_8 False
19 batch_normalization_8 False
20 activation_8 False
21 conv2d_6 False
22 conv2d_9 False
23 batch_normalization_6 False
24 batch_normalization_9 False
25 activation_6 False
26 activation_9 False
27 average_pooling2d False
28 conv2d_5 False
29 conv2d_7 False
30 conv2d_10 False
31 conv2d_11 False
32 batch_normalization_5 False
33 batch_normalization_7 False
34 batch_normalization_10 False
35 batch_normalization_11 False
36 activation_5 False
37 activation_7 False
38 activation_10 False
39 activation_11 False
40 mixed_5b False
41 conv2d_15 False
42 batch_normalization_15 Fal

In [17]:
c = [tfk.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)]

history = model.fit(
    x = train_gen,
    batch_size = batch_size,
    epochs = 200,
    validation_data = valid_gen,
    class_weight = class_weight,
    callbacks = c
).history

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/200
194/194 [==============================] - 1715s 9s/step - loss: 0.0741 - accuracy: 0.7803 - val_loss: 11.4816 - val_accuracy: 0.6159
Epoch 2/200
194/194 [==============================] - 1648s 9s/step - loss: 0.0310 - accuracy: 0.8784 - val_loss: 1.1311 - val_accuracy: 0.7978
Epoch 3/200
194/194 [==============================] - 1648s 9s/step - loss: 0.0283 - accuracy: 0.8877 - val_loss: 6.0352 - val_accuracy: 0.8005
Epoch 4/200
194/194 [==============================] - 1645s 8s/step - loss: 0.0259 - accuracy: 0.8945 - val_loss: 0.5666 - val_accuracy: 0.8510
Epoch 5/200
194/194 [==============================] - 1646s 8s/step - loss: 0.0204 - accuracy: 0.9129 - val_loss: 0.3684 - val_accuracy: 0.8535
Epoch 6/200
194/194 [==============================] - 1650s 9s/step - loss: 0.0181 - accuracy: 0.9199 - val_loss: 0.2122 - val_accuracy

In [18]:
model.save('./' + model_dir)

INFO:tensorflow:Assets written to: ./incepti_resnet_v2_divided2_ds\assets
